<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

#!pip install onnx onnxruntime


In [1]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name = "X"
softmax_output_name = "SoftmaxOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensor
#-------------------------------------------------------------------------------

# Create 2-rank input tensor
input_tensor = onnx.helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, [None, None])

# Create output tensor (final result after softmax operation)
softmax_output = onnx.helper.make_tensor_value_info(softmax_output_name, onnx.TensorProto.FLOAT, [None, None])

# Define Softmax node: Y = Softmax(X) with axis=1
softmax_node = onnx.helper.make_node("Softmax", [input_name], [softmax_output_name], axis=1)

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[softmax_node],
    name="Softmax",
    inputs=[input_tensor],
    outputs=[softmax_output]
)

# Create the ONNX model (Softmax opset 13)
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)])
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "softmax.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("softmax.onnx")

def do_softmax(x):
    # Run inference
    output = session.run(None, {input_name: x})

    x_f = np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '')
    o_f = np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '')

    # Display results
    print(f"X={x_f}")
    print(f"Result: softmax(X)={o_f}")

np.set_printoptions(precision=8, floatmode='fixed')


#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 1x2 vector
x = np.array([[9.5, 35.7]], dtype=np.float32)
do_softmax(x)

# Case N2: 2x3 matrix
x = np.array([[1.0, 2.0, 3.0],
              [1.0, 2.0, 3.0]], dtype=np.float32)
do_softmax(x)


#--------------------------------------------------------------------
# Non nominal cases (nan, +inf, -inf)
#--------------------------------------------------------------------

v = [np.nan, np.inf, -np.inf]

for x_val in v:
    x_np = np.array([[x_val, 0.0, 1.0],
                     [x_val, 2.0, 3.0]], dtype=np.float32)
    do_softmax(x_np)


#--------------------------------------------------------------------
# Examples from documentation
#--------------------------------------------------------------------

print("\n## Example 1")

# Example 1
# X = [9.5, 35.7]
x_example_1 = np.array([[9.5, 35.7]], dtype=np.float32)
do_softmax(x_example_1)

print("\n## Example 2")

# Example 2
# X =
# [[1 2 3]
#  [1 2 3]]
x_example_2 = np.array([
    [1.0, 2.0, 3.0],
    [1.0, 2.0, 3.0]
], dtype=np.float32)
do_softmax(x_example_2)

print("\n## Example 3")

# Example 3: slice with +inf
x_example_3 = np.array([[1.0, np.inf, 2.0]], dtype=np.float32)
do_softmax(x_example_3)

print("\n## Example 4")

# Example 4: slice with -inf
x_example_4 = np.array([[-np.inf, -np.inf]], dtype=np.float32)
do_softmax(x_example_4)

print("\n## Example 5")

# Example 5: slice with NaN
x_example_5 = np.array([[1.0, np.nan, 2.0]], dtype=np.float32)
do_softmax(x_example_5)

X=[[ 9.50000000,35.70000076]]
Result: softmax(X)=[[4.18296517e-12,1.00000000e+00]]
X=[[1.00000000,2.00000000,3.00000000], [1.00000000,2.00000000,3.00000000]]
Result: softmax(X)=[[0.09003057,0.24472848,0.66524094], [0.09003057,0.24472848,0.66524094]]
X=[[       nan,0.00000000,1.00000000], [       nan,2.00000000,3.00000000]]
Result: softmax(X)=[[nan,nan,nan], [nan,nan,nan]]
X=[[       inf,0.00000000,1.00000000], [       inf,2.00000000,3.00000000]]
Result: softmax(X)=[[nan,nan,nan], [nan,nan,nan]]
X=[[      -inf,0.00000000,1.00000000], [      -inf,2.00000000,3.00000000]]
Result: softmax(X)=[[0.00000000,0.26894143,0.73105860], [0.00000000,0.26894143,0.73105860]]

## Example 1
X=[[ 9.50000000,35.70000076]]
Result: softmax(X)=[[4.18296517e-12,1.00000000e+00]]

## Example 2
X=[[1.00000000,2.00000000,3.00000000], [1.00000000,2.00000000,3.00000000]]
Result: softmax(X)=[[0.09003057,0.24472848,0.66524094], [0.09003057,0.24472848,0.66524094]]

## Example 3
X=[[1.00000000,       inf,2.00000000]]
Re

In [2]:
#--------------------------------------------------------------------
# Additional special cases (tests 6 to 20)
#--------------------------------------------------------------------

print("\n## Test 6: 3 flottants normaux + +inf")
x_test_6 = np.array([[1.0, 2.0, 3.0, np.inf]], dtype=np.float32)
do_softmax(x_test_6)

print("\n## Test 7: 3 flottants normaux + -inf")
x_test_7 = np.array([[1.0, 2.0, 3.0, -np.inf]], dtype=np.float32)
do_softmax(x_test_7)

print("\n## Test 8: 3 flottants normaux + NaN")
x_test_8 = np.array([[1.0, 2.0, 3.0, np.nan]], dtype=np.float32)
do_softmax(x_test_8)

print("\n## Test 9: 3 flottants égaux à -0")
x_test_9 = np.array([[-0.0, -0.0, -0.0]], dtype=np.float32)
do_softmax(x_test_9)

print("\n## Test 10: 3 flottants -0, 0, 0")
x_test_10 = np.array([[-0.0, 0.0, 0.0]], dtype=np.float32)
do_softmax(x_test_10)

print("\n## Test 11: 3 flottants normaux + +inf et -inf")
x_test_11 = np.array([[1.0, 2.0, 3.0, np.inf, -np.inf]], dtype=np.float32)
do_softmax(x_test_11)

print("\n## Test 12: 3 flottants normaux + +inf et NaN")
x_test_12 = np.array([[1.0, 2.0, 3.0, np.inf, np.nan]], dtype=np.float32)
do_softmax(x_test_12)

print("\n## Test 13: 3 flottants normaux + +inf et +0")
x_test_13 = np.array([[1.0, 2.0, 3.0, np.inf, 0.0]], dtype=np.float32)
do_softmax(x_test_13)

print("\n## Test 14: 3 flottants normaux + +inf et -0")
x_test_14 = np.array([[1.0, 2.0, 3.0, np.inf, -0.0]], dtype=np.float32)
do_softmax(x_test_14)

print("\n## Test 15: 3 flottants normaux + -inf et NaN")
x_test_15 = np.array([[1.0, 2.0, 3.0, -np.inf, np.nan]], dtype=np.float32)
do_softmax(x_test_15)

print("\n## Test 16: 3 flottants normaux + -inf et +0")
x_test_16 = np.array([[1.0, 2.0, 3.0, -np.inf, 0.0]], dtype=np.float32)
do_softmax(x_test_16)

print("\n## Test 17: 3 flottants normaux + -inf et -0")
x_test_17 = np.array([[1.0, 2.0, 3.0, -np.inf, -0.0]], dtype=np.float32)
do_softmax(x_test_17)

print("\n## Test 18: 3 flottants normaux <0 + -0 + +0")
x_test_18 = np.array([[-1.0, -2.0, -3.0, -0.0, 0.0]], dtype=np.float32)
do_softmax(x_test_18)

print("\n## Test 19: 3 flottants normaux >0 + -0 + +0")
x_test_19 = np.array([[1.0, 2.0, 3.0, -0.0, 0.0]], dtype=np.float32)
do_softmax(x_test_19)

print("\n## Test 20: 1 flottant >0, 1 flottant <0, + -0, +0")
x_test_20 = np.array([[1.0, -1.0, -0.0, 0.0, 2.0]], dtype=np.float32)
do_softmax(x_test_20)


## Test 6: 3 flottants normaux + +inf
X=[[1.00000000,2.00000000,3.00000000,       inf]]
Result: softmax(X)=[[nan,nan,nan,nan]]

## Test 7: 3 flottants normaux + -inf
X=[[1.00000000,2.00000000,3.00000000,      -inf]]
Result: softmax(X)=[[0.09003057,0.24472848,0.66524094,0.00000000]]

## Test 8: 3 flottants normaux + NaN
X=[[1.00000000,2.00000000,3.00000000,       nan]]
Result: softmax(X)=[[nan,nan,nan,nan]]

## Test 9: 3 flottants égaux à -0
X=[[-0.00000000,-0.00000000,-0.00000000]]
Result: softmax(X)=[[0.33333334,0.33333334,0.33333334]]

## Test 10: 3 flottants -0, 0, 0
X=[[-0.00000000, 0.00000000, 0.00000000]]
Result: softmax(X)=[[0.33333334,0.33333334,0.33333334]]

## Test 11: 3 flottants normaux + +inf et -inf
X=[[1.00000000,2.00000000,3.00000000,       inf,      -inf]]
Result: softmax(X)=[[nan,nan,nan,nan,nan]]

## Test 12: 3 flottants normaux + +inf et NaN
X=[[1.00000000,2.00000000,3.00000000,       inf,       nan]]
Result: softmax(X)=[[nan,nan,nan,nan,nan]]

## Test 13: 3 flotta